# CERRA snow-depth climatology

## Import packages

In [ ]:
import calendar

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from c3s_eqc_automatic_quality_control import download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Select climatology periods
years_start = [1986, 1990]
years_stop = [2016, 2020]
assert len(years_start) == len(years_stop)

# Select month of interest
month = 1
assert month in range(1, 13), f"{month=} is invalid"

# Select region of interest
region = "Alps"
region_slices = {
    "Alps": {"lon_slice": slice(5, 15), "lat_slice": slice(48, 43)},
    "Scandinavia": {"lon_slice": slice(10, 42), "lat_slice": slice(72, 62)},
    "Lazio": {"lon_slice": slice(11, 14), "lat_slice": slice(43, 41)},
    "Italy": {"lon_slice": slice(6, 19), "lat_slice": slice(48, 36)},
}
assert region in region_slices, f"{region=} is invalid"

## Define request

In [ ]:
collection_id = "reanalysis-cerra-single-levels"
request = {
    "variable": "snow_depth",
    "level_type": "surface_or_atmosphere",
    "data_type": "reanalysis",
    "product_type": "analysis",
    "time": [f"{i:02d}:00" for i in range(0, 24, 3)],
    "format": "grib",
}

## Functions to cache

In [ ]:
def regionalised_max(ds, lon_slice, lat_slice):
    time = ds["forecast_reference_time"].mean()
    ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)
    ds = ds.max("forecast_reference_time", keep_attrs=True)
    return ds.expand_dims(time=[time.values])

## Download and transform

In [ ]:
dataarrays = []
for year_start, year_stop in zip(years_start, years_stop):
    requests = []
    for year in range(year_start, year_stop + 1):
        time_request = {
            "year": year,
            "month": month,
            "day": list(range(1, calendar.monthrange(year, month)[1] + 1)),
        }
        requests.append(request | time_request)
    ds = download.download_and_transform(
        collection_id,
        requests,
        transform_func=regionalised_max,
        transform_func_kwargs=region_slices[region],
    )
    (varname,) = ds.data_vars
    da = ds[varname].max("time", keep_attrs=True)
    dataarrays.append(da.expand_dims(period=[f"{year_start}-{year_stop}"]))
da = xr.concat(dataarrays, "period")

## Plot maps

In [ ]:
lon_slice = region_slices[region]["lon_slice"]
lat_slice = region_slices[region]["lat_slice"]
projection = ccrs.LambertConformal(
    central_longitude=(lon_slice.start + lon_slice.stop) / 2,
    central_latitude=(lat_slice.start + lat_slice.stop) / 2,
)

facet_obj = plot.projected_map(
    da, projection=projection, col="period", cmap="CMRmap_r", robust=True
)
for i, ax in enumerate(facet_obj.axs.flatten()):
    gl = ax.gridlines(x_inline=False)
    gl.bottom_labels = True
    gl.left_labels = False if i else True
    gl.xlabel_style = {"rotation": 0}
_ = plt.suptitle(calendar.month_name[month])

## Plot pdf

In [ ]:
bins = np.linspace(da.min(), da.max(), 30)
hist_dataarrays = []
for period, da_period in da.groupby("period"):
    hist, _ = np.histogram(da_period, bins=bins, density=True)
    hist_dataarrays.append(xr.DataArray(hist, dims="bins").expand_dims(period=[period]))
da_hist = xr.concat(hist_dataarrays, "period")
da_hist = da_hist.assign_coords(bins=(bins[1:] + bins[:-1]) / 2)
da_hist["bins"].attrs = da.attrs
da_hist.attrs["long_name"] = "Probability Density"

fig, ax = plt.subplots()
ax.set_prop_cycle(color=["darkorange", "red"])
da_hist.plot(hue="period", yscale="log")
_ = plt.title(calendar.month_name[month])